## Introduction

KerasNLP provides building blocks for NLP (model layers, tokenizers, metrics, etc.) and
makes it convenient to construct NLP pipelines.

In this example, we'll use KerasNLP layers to build an encoder-decoder Transformer
model, and train it on the English-to-Spanish machine translation task.

You'll learn how to:

- Tokenize text using `keras_nlp.tokenizers.WordPieceTokenizer`.
- Implement a sequence-to-sequence Transformer model using KerasNLP's
`keras_nlp.layers.TransformerEncoder`, `keras_nlp.layers.TransformerDecoder` and
`keras_nlp.layers.TokenAndPositionEmbedding` layers, and train it.
- Use `keras_nlp.samplers` to generate translations of unseen input sentences
 using the top-p decoding strategy!

Don't worry if you aren't familiar with KerasNLP. This tutorial will start with
the basics. Let's dive right in!

## Setup

Before we start implementing the pipeline, let's import all the libraries we need.

In [ ]:
!pip install -q --upgrade keras-nlp
!pip install -q --upgrade keras  # Upgrade to Keras 3.
!pip install -q tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 101.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 67.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatibl

In [ ]:
import keras_nlp
import pathlib
import random

import keras
from keras import ops

import tensorflow.data as tf_data
from tensorflow_text.tools.wordpiece_vocab import (
    bert_vocab_from_dataset as bert_vocab,
)

Let's also define our parameters/hyperparameters.

In [ ]:
BATCH_SIZE = 64
EPOCHS = 10  # This should be at least 10 for convergence
MAX_SEQUENCE_LENGTH = 100
ENG_VOCAB_SIZE = 50000
INDO_VOCAB_SIZE = 50000

EMBED_DIM = 256
INTERMEDIATE_DIM = 2048
NUM_HEADS = 8

## Downloading the data

We'll be working with an English-to-Spanish translation dataset
provided by [Anki](https://www.manythings.org/anki/). Let's download it:

### my their dataset

In [ ]:
text_file = keras.utils.get_file(
    fname="spa-eng.zip",
    origin="http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip",
    extract=True,
)
text_file = pathlib.Path(text_file).parent / "spa-eng" / "spa.txt"

### my dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/gabungan_english_indo.csv", encoding='latin1')
df

,English,Indonesian
0,Hi.,Hai.
1,Run!,Lari!
2,Run.,Lari!
3,Who?,Siapa?
4,Wow!,Wow!
...,...,...
37586,Only 0.028 g of some snakes? venom is strong e...,"Bisa ular sebanyak 0,028 gram cukup untuk memb..."
37587,It returned five of them and their families in...,Pyongyang mengembalikan lima dari mereka dan k...
37588,The increase was expected to be fueled by net ...,Dalam sebuah pernyataannya kepada Bursa Efek J...
37589,Four Pakistani paramilitary soldiers were inju...,"Empat orang prajurit militer Pakistan, Minggu,..."


In [ ]:
df.drop(columns=['Javanese'], inplace=True)
df

,English,Indonesian
0,Hi.,Hai.
1,Run!,Lari!
2,Run.,Lari!
3,Who?,Siapa?
4,Wow!,Wow!
...,...,...
13565,"When she was finished ironing, Mary switched o...","Ketika dia sudah selesai menyetrika, Mary mema..."
13566,"When he got up to make his speech, Tom's mind ...","Ketika Tom berdiri untuk pidato, pikirannya ti..."
13567,"January, February, March, April, May, June, Ju...","Januari, Februari, Maret, April, Mei, Juni, Ju..."
13568,"Irene Pepperberg, a researcher at Northwestern...","Irene Pepperberg, seorang peneliti di Universi..."


In [ ]:
output_file_path = '/content/english-indo.txt'
df.to_csv(output_file_path, sep="\t", index=False, header=False)

## Parsing the data

Each line contains an English sentence and its corresponding Spanish sentence.
The English sentence is the *source sequence* and Spanish one is the *target sequence*.
Before adding the text to a list, we convert it to lowercase.

In [ ]:
import pathlib

text_file = pathlib.Path("/content/english-indo.txt")

In [ ]:
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, indo = line.split("\t")
    eng = eng.lower()
    indo = indo.lower()
    text_pairs.append((eng, indo))

Here's what our sentence pairs look like:

In [ ]:
for _ in range(5):
    print(random.choice(text_pairs))

('we also often hear the electric turning off in a region related to an occurrence in the transformer because that is electric distribution center around the area.', 'kita juga sering mendengar pemadaman listrik di suatu daerah dihubungkan dengan kejadian di suatu gardu, karena memang di gardu inilah pusat penyaluran listrik di daerah tersebut.')
("tom says he isn't guilty.", 'tom bilang dia tidak bersalah.')
('the broad-based rise also stemmed from firm u.s. retail sales for march, commerce department data showed retail sales growing a stronger-than-expected 0.7 percent in march.', 'sebuah laporan pemerintah memperlihatkan bahwa penjualan ritel maret, sebagai kajian salah satu indikator kemakmuran ekonomi, melonjak lebih besar dari perkiraan 0,7 persen juga disambut positif para investor.')
("wow! it's been a long time.", 'wow! sudah sekian lama.')
('according to the statement, some $41 million of the funds will go to a program to empower farmers through a farmers group capacity devel

Now, let's split the sentence pairs into a training set, a validation set,
and a test set.

In [ ]:
import random

# Assuming text_pairs is defined elsewhere
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")


37591 total pairs
31953 training pairs
5638 validation pairs


## Tokenizing the data

We'll define two tokenizers - one for the source language (English), and the other
for the target language (Spanish). We'll be using
`keras_nlp.tokenizers.WordPieceTokenizer` to tokenize the text.
`keras_nlp.tokenizers.WordPieceTokenizer` takes a WordPiece vocabulary
and has functions for tokenizing the text, and detokenizing sequences of tokens.

Before we define the two tokenizers, we first need to train them on the dataset
we have. The WordPiece tokenization algorithm is a subword tokenization algorithm;
training it on a corpus gives us a vocabulary of subwords. A subword tokenizer
is a compromise between word tokenizers (word tokenizers need very large
vocabularies for good coverage of input words), and character tokenizers
(characters don't really encode meaning like words do). Luckily, KerasNLP
makes it very simple to train WordPiece on a corpus with the
`keras_nlp.tokenizers.compute_word_piece_vocabulary` utility.

In [ ]:

def train_word_piece(text_samples, vocab_size, reserved_tokens):
    word_piece_ds = tf_data.Dataset.from_tensor_slices(text_samples)
    vocab = keras_nlp.tokenizers.compute_word_piece_vocabulary(
        word_piece_ds.batch(1000).prefetch(2),
        vocabulary_size=vocab_size,
        reserved_tokens=reserved_tokens,
    )
    return vocab


Every vocabulary has a few special, reserved tokens. We have four such tokens:

- `"[PAD]"` - Padding token. Padding tokens are appended to the input sequence
length when the input sequence length is shorter than the maximum sequence length.
- `"[UNK]"` - Unknown token.
- `"[START]"` - Token that marks the start of the input sequence.
- `"[END]"` - Token that marks the end of the input sequence.

In [ ]:
reserved_tokens = ["[PAD]", "[UNK]", "[START]", "[END]"]

eng_samples = [text_pair[0] for text_pair in train_pairs]
eng_vocab = train_word_piece(eng_samples, ENG_VOCAB_SIZE, reserved_tokens)

indo_samples = [text_pair[1] for text_pair in train_pairs]
indo_vocab = train_word_piece(indo_samples, INDO_VOCAB_SIZE, reserved_tokens)

Let's see some tokens!

In [ ]:
print("English Tokens: ", eng_vocab)
print("Indo Tokens: ", indo_vocab)

English Tokens:  ['[PAD]', '[UNK]', '[START]', '[END]', '!', '"', '$', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '?', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¢', '©', '°', 'â', 'ã', 'the', 'to', 'of', 'in', 'and', 'is', 'on', 'said', '##s', 'that', 'for', 'was', 'he', 'with', 'it', 'by', 'at', 'from', 'as', 'this', 'be', 'you', 'tom', 'are', 'have', 'has', 'not', 'percent', 'will', 'an', 'which', 'we', 'were', 'had', 'his', 'its', '##ing', 'but', 'year', 'us', 'they', 'would', 'after', 'can', 'their', 'bank', 'one', 'who', '##ed', 'also', 'been', 'two', 'government', 'or', 'first', '##d', 'up', 'there', 'when', 'do', 'last', 'indonesia', 'more', 'time', 'what', 'she', '##a', 'state', 'all', '##e', '##ly', 'other', 'out', 'so', 'my', 'people', 'only', 'than', 'trillion', 'three', 'about', 'here', 'like', 'no', 'over

Now, let's define the tokenizers. We will configure the tokenizers with the
the vocabularies trained above.

In [ ]:
import json

with open('indo_vocab.json', 'w') as f:
    json.dump(indo_vocab, f)

with open('eng_vocab.json', 'w') as f:
    json.dump(eng_vocab, f)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!rm "/content/gdrive/MyDrive/TensorflowModels/eng_vocab.json"
!rm "/content/gdrive/MyDrive/TensorflowModels/indo_vocab.json"

In [ ]:
!cp "/content/eng_vocab.json" "/content/gdrive/MyDrive/TensorflowModels"
!cp "/content/indo_vocab.json" "/content/gdrive/MyDrive/TensorflowModels"

file jsonnya ada di drive gw
https://drive.google.com/drive/folders/1YaHN2nk7i5OYmGgJ6SGGLglUPAKpaFDY?usp=drive_link

cek dari sini gar buat tokenize dari json
https://medium.com/@andrew.w.sale/deploying-text-classification-from-keras-to-tensorflow-js-92c614dca4ec

In [ ]:
eng_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=eng_vocab, lowercase=False
)
indo_tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary=indo_vocab, lowercase=False
)

In [ ]:
import json

with open('indo_vocab.json', 'w') as f:
    json.dump(indo_vocab, f)

with open('eng_vocab.json', 'w') as f:
    json.dump(eng_vocab, f)

Let's try and tokenize a sample from our dataset! To verify whether the text has
been tokenized correctly, we can also detokenize the list of tokens back to the
original text.

In [ ]:
eng_input_ex = text_pairs[0][0]
eng_tokens_ex = eng_tokenizer.tokenize(eng_input_ex)
print("English sentence: ", eng_input_ex)
print("Tokens: ", eng_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    eng_tokenizer.detokenize(eng_tokens_ex),
)

print()

indo_input_ex = text_pairs[0][1]
indo_tokens_ex = indo_tokenizer.tokenize(indo_input_ex)
print("indo sentence: ", indo_input_ex)
print("Tokens: ", indo_tokens_ex)
print(
    "Recovered text after detokenizing: ",
    indo_tokenizer.detokenize(indo_tokens_ex),
)

English sentence:  turn to the right.
Tokens:  tf.Tensor([878  66  65 356  15], shape=(5,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'turn to the right .', shape=(), dtype=string)

indo sentence:  belok ke kanan.
Tokens:  tf.Tensor([  36  812 1222   79 1609   16], shape=(6,), dtype=int32)
Recovered text after detokenizing:  tf.Tensor(b'belok ke kanan .', shape=(), dtype=string)


## Format datasets

Next, we'll format our datasets.

At each training step, the model will seek to predict target words N+1 (and beyond)
using the source sentence and the target words 0 to N.

As such, the training dataset will yield a tuple `(inputs, targets)`, where:

- `inputs` is a dictionary with the keys `encoder_inputs` and `decoder_inputs`.
`encoder_inputs` is the tokenized source sentence and `decoder_inputs` is the target
sentence "so far",
that is to say, the words 0 to N used to predict word N+1 (and beyond) in the target
sentence.
- `target` is the target sentence offset by one step:
it provides the next words in the target sentence -- what the model will try to predict.

We will add special tokens, `"[START]"` and `"[END]"`, to the input Spanish
sentence after tokenizing the text. We will also pad the input to a fixed length.
This can be easily done using `keras_nlp.layers.StartEndPacker`.

In [ ]:

def preprocess_batch(eng, indo):
    batch_size = ops.shape(indo)[0]

    eng = eng_tokenizer(eng)
    indo = indo_tokenizer(indo)

    # Pad `eng` to `MAX_SEQUENCE_LENGTH`.
    eng_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH,
        pad_value=eng_tokenizer.token_to_id("[PAD]"),
    )
    eng = eng_start_end_packer(eng)

    # Add special tokens (`"[START]"` and `"[END]"`) to `indo` and pad it as well.
    indo_start_end_packer = keras_nlp.layers.StartEndPacker(
        sequence_length=MAX_SEQUENCE_LENGTH + 1,
        start_value=indo_tokenizer.token_to_id("[START]"),
        end_value=indo_tokenizer.token_to_id("[END]"),
        pad_value=indo_tokenizer.token_to_id("[PAD]"),
    )
    indo = indo_start_end_packer(indo)

    return (
        {
            "encoder_inputs": eng,
            "decoder_inputs": indo[:, :-1],
        },
        indo[:, 1:],
    )


def make_dataset(pairs):
    eng_texts, indo_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    indo_texts = list(indo_texts)
    dataset = tf_data.Dataset.from_tensor_slices((eng_texts, indo_texts))
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.map(preprocess_batch, num_parallel_calls=tf_data.AUTOTUNE)
    return dataset.cache().shuffle(2048).prefetch(16)


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

Let's take a quick look at the sequence shapes
(we have batches of 64 pairs, and all sequences are 40 steps long):

In [ ]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")


inputs["encoder_inputs"].shape: (64, 100)
inputs["decoder_inputs"].shape: (64, 100)
targets.shape: (64, 100)


## Building the model

Now, let's move on to the exciting part - defining our model!
We first need an embedding layer, i.e., a vector for every token in our input sequence.
This embedding layer can be initialised randomly. We also need a positional
embedding layer which encodes the word order in the sequence. The convention is
to add these two embeddings. KerasNLP has a `keras_nlp.layers.TokenAndPositionEmbedding `
layer which does all of the above steps for us.

Our sequence-to-sequence Transformer consists of a `keras_nlp.layers.TransformerEncoder`
layer and a `keras_nlp.layers.TransformerDecoder` layer chained together.

The source sequence will be passed to `keras_nlp.layers.TransformerEncoder`, which
will produce a new representation of it. This new representation will then be passed
to the `keras_nlp.layers.TransformerDecoder`, together with the target sequence
so far (target words 0 to N). The `keras_nlp.layers.TransformerDecoder` will
then seek to predict the next words in the target sequence (N+1 and beyond).

A key detail that makes this possible is causal masking.
The `keras_nlp.layers.TransformerDecoder` sees the entire sequence at once, and
thus we must make sure that it only uses information from target tokens 0 to N
when predicting token N+1 (otherwise, it could use information from the future,
which would result in a model that cannot be used at inference time). Causal masking
is enabled by default in `keras_nlp.layers.TransformerDecoder`.

We also need to mask the padding tokens (`"[PAD]"`). For this, we can set the
`mask_zero` argument of the `keras_nlp.layers.TokenAndPositionEmbedding` layer
to True. This will then be propagated to all subsequent layers.

In [ ]:
# Encoder
encoder_inputs = keras.Input(batch_shape=(None,MAX_SEQUENCE_LENGTH,), name="encoder_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=ENG_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(encoder_inputs)

encoder_outputs = keras_nlp.layers.TransformerEncoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(inputs=x)
encoder = keras.Model(encoder_inputs, encoder_outputs)


# Decoder
decoder_inputs = keras.Input(batch_shape=(None, MAX_SEQUENCE_LENGTH,), name="decoder_inputs")
encoded_seq_inputs = keras.Input(batch_shape=(None, None, EMBED_DIM), name="decoder_state_inputs")

x = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=INDO_VOCAB_SIZE,
    sequence_length=MAX_SEQUENCE_LENGTH,
    embedding_dim=EMBED_DIM,
)(decoder_inputs)

x = keras_nlp.layers.TransformerDecoder(
    intermediate_dim=INTERMEDIATE_DIM, num_heads=NUM_HEADS
)(decoder_sequence=x, encoder_sequence=encoded_seq_inputs)
x = keras.layers.Dropout(0.5)(x)
decoder_outputs = keras.layers.Dense(INDO_VOCAB_SIZE, activation="softmax")(x)
decoder = keras.Model(
    [
        decoder_inputs,
        encoded_seq_inputs,
    ],
    decoder_outputs,
)
decoder_outputs = decoder([decoder_inputs, encoder_outputs])

transformer = keras.Model(
    [encoder_inputs, decoder_inputs],
    decoder_outputs,
    name="transformer",
)

## Training our model

We'll use accuracy as a quick way to monitor training progress on the validation data.
Note that machine translation typically uses BLEU scores as well as other metrics,
rather than accuracy. However, in order to use metrics like ROUGE, BLEU, etc. we
will have decode the probabilities and generate the text. Text generation is
computationally expensive, and performing this during training is not recommended.

Here we only train for 1 epoch, but to get the model to actually converge
you should train for at least 10 epochs.

In [ ]:
transformer.summary()
transformer.compile(
    "adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

Model: "transformer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ encoder_inputs            │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_and_position_embed… │ (None, 100, 256)       │      7,705,600 │ encoder_inputs[0][0]   │
│ (TokenAndPositionEmbeddi… │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ decoder_inputs            │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ transformer_encoder       │ (None, 100, 256)       │      1,315,072 │ token_and_position_em… │
│ (TransformerEncoder)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional_3 (Functional) │ (None, 100, 30000)     │     16,994,352 │ decoder_inputs[0][0],  │
│                           │                        │                │ transformer_encoder[0… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 26,015,024 (99.24 MB)

 Trainable params: 26,015,024 (99.24 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Epoch 1/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 188s 346ms/step - accuracy: 0.8011 - loss: 2.1494 - val_accuracy: 0.8177 - val_loss: 1.1706
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 165ms/step - accuracy: 0.8222 - loss: 1.1306 - val_accuracy: 0.8251 - val_loss: 1.0593
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 165ms/step - accuracy: 0.8297 - loss: 1.0193 - val_accuracy: 0.8329 - val_loss: 0.9849
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 166ms/step - accuracy: 0.8383 - loss: 0.9343 - val_accuracy: 0.8391 - val_loss: 0.9277
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 165ms/step - accuracy: 0.8480 - loss: 0.8505 - val_accuracy: 0.8457 - val_loss: 0.8838
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 142s 165ms/step - accuracy: 0.8548 - loss: 0.7951 - val_accuracy: 0.8510 - val_loss: 0.8502
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 164ms/step - accuracy: 0.8630 - loss: 0.7365 - val_accuracy: 0.8554 - val_loss: 0.8272
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 82s 165ms/step - accuracy: 0.8692 - loss

## Decoding test sentences (qualitative analysis)

Finally, let's demonstrate how to translate brand new English sentences.
We simply feed into the model the tokenized English sentence
as well as the target token `"[START]"`. The model outputs probabilities of the
next token. We then we repeatedly generated the next token conditioned on the
tokens generated so far, until we hit the token `"[END]"`.

For decoding, we will use the `keras_nlp.samplers` module from
KerasNLP. Greedy Decoding is a text decoding method which outputs the most
likely next token at each time step, i.e., the token with the highest probability.

In [ ]:

def decode_sequences(input_sentences):
    batch_size = 1

    # Tokenize the encoder input.
    encoder_input_tokens = ops.convert_to_tensor(eng_tokenizer(input_sentences))
    if len(encoder_input_tokens[0]) < MAX_SEQUENCE_LENGTH:
        pads = ops.full((1, MAX_SEQUENCE_LENGTH - len(encoder_input_tokens[0])), 0)
        encoder_input_tokens = ops.concatenate(
            [encoder_input_tokens.to_tensor(), pads], 1
        )

    # Define a function that outputs the next token's probability given the
    # input sequence.
    def next(prompt, cache, index):
        logits = transformer([encoder_input_tokens, prompt])[:, index - 1, :]
        # Ignore hidden states for now; only needed for contrastive search.
        hidden_states = None
        return logits, hidden_states, cache

    # Build a prompt of length 40 with a start token and padding tokens.
    length = 100
    start = ops.full((batch_size, 1), indo_tokenizer.token_to_id("[START]"))
    pad = ops.full((batch_size, length - 1), indo_tokenizer.token_to_id("[PAD]"))
    prompt = ops.concatenate((start, pad), axis=-1)

    generated_tokens = keras_nlp.samplers.GreedySampler()(
        next,
        prompt,
        stop_token_ids=[indo_tokenizer.token_to_id("[END]")],
        index=1,  # Start sampling after start token.
    )
    generated_sentences = indo_tokenizer.detokenize(generated_tokens)
    return generated_sentences

input_sentence = "how much?"
translated = decode_sequences([input_sentence])
translated = translated.numpy()[0].decode("utf-8")
translated = (
    translated.replace("[PAD]", "")
    .replace("[START]", "")
    .replace("[END]", "")
    .strip()
)
print(input_sentence)
print(translated)
print()

how much?
berapa harganya ?



## Evaluating our model (quantitative analysis)

There are many metrics which are used for text generation tasks. Here, to
evaluate translations generated by our model, let's compute the ROUGE-1 and
ROUGE-2 scores. Essentially, ROUGE-N is a score based on the number of common
n-grams between the reference text and the generated text. ROUGE-1 and ROUGE-2
use the number of common unigrams and bigrams, respectively.

We will calculate the score over 30 test samples (since decoding is an
expensive process).

In [ ]:
rouge_1 = keras_nlp.metrics.RougeN(order=1)
rouge_2 = keras_nlp.metrics.RougeN(order=2)

for test_pair in test_pairs[:30]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]

    translated_sentence = decode_sequences([input_sentence])
    translated_sentence = translated_sentence.numpy()[0].decode("utf-8")
    translated_sentence = (
        translated_sentence.replace("[PAD]", "")
        .replace("[START]", "")
        .replace("[END]", "")
        .strip()
    )

    rouge_1(reference_sentence, translated_sentence)
    rouge_2(reference_sentence, translated_sentence)

print("ROUGE-1 Score: ", rouge_1.result())
print("ROUGE-2 Score: ", rouge_2.result())

ROUGE-1 Score:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.5457408>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.49427488>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.5112927>}
ROUGE-2 Score:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.27380952>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.25079364>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.257325>}


After 10 epochs, the scores are as follows:

|               | **ROUGE-1** | **ROUGE-2** |
|:-------------:|:-----------:|:-----------:|
| **Precision** |    0.568    |    0.374    |
|   **Recall**  |    0.615    |    0.394    |
|  **F1 Score** |    0.579    |    0.381    |

In [ ]:
transformer.save('english-indo.keras')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('/content/english-indo.h5')
tf.saved_model.save(model, '/content/my_saved_model')

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model "/content/my_saved_model" "/content/my_tfjs_model"

2024-06-13 17:49:12.726490: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-06-13 17:49:15.555347: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:47] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2024-06-13 17:49:15.813088: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
2024-06-13 17:49:15.856616: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
2024-06-13 17:49:15.894222: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
2024-06-13 17:49:16.124107: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30720000 exceeds 10% of free system memory.
2024-06-13 17:49:16.202605: W external/local_tsl/tsl/framework/cpu_alloc

In [ ]:
!cp -r "/content/my_tfjs_model" "/content/gdrive/MyDrive/TensorflowModels"

In [ ]:
!cp "/content/indo_vocab.json" "/content/gdrive/MyDrive/TensorflowModels"
!cp "/content/eng_vocab.json" "/content/gdrive/MyDrive/TensorflowModels"

In [ ]:
from google.colab import files

files.download('example.txt')